In [1]:
# Setup
!git clone https://github.com/openreview/openreview-py.git
!pip install -e ./openreview-py

Cloning into 'openreview-py'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 5229 (delta 259), reused 247 (delta 212), pack-reused 4908
Receiving objects: 100% (5229/5229), 4.10 MiB | 6.14 MiB/s, done.
Resolving deltas: 100% (3826/3826), done.
Obtaining file:///Users/cmondragonch/Documents/openreview-expertise/expertise/openreview-py
  Attempting uninstall: openreview-py
    Found existing installation: openreview-py 1.0.10
    Uninstalling openreview-py-1.0.10:
      Successfully uninstalled openreview-py-1.0.10
  Running setup.py develop for openreview-py


In [1]:
import json, argparse
from pathlib import Path
from itertools import chain
import openreview
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict

class OpenReviewExpertise(object):
    def __init__(self, openreview_client, config):
        self.openreview_client = openreview_client
        self.config = config
        self.root = Path(config['dataset']['directory'])

        self.metadata = {
            'submission_count': 0,
            'no_publications_count': 0,
            'no_publications': []
        }

    def convert_to_list(self, config_invitations):
        if (isinstance(config_invitations, str)):
            invitations = [config_invitations]
        else:
            invitations = config_invitations
        assert isinstance(invitations, list), 'Input should be a str or a list'
        return invitations

    def get_publications(self, author_id):
        content = {
            'authorids': author_id
        }

        publications = openreview.tools.iterget_notes(self.openreview_client, content=content)

        # Get all publications and assign tcdate to cdate in case cdate is None. If tcdate is also None
        # assign cdate := 0
        unsorted_publications = []
        for publication in publications:
            # Check if paper has abstract or title, otherwise continue
            if self.config.get('dataset', {}).get('with_abstract', False):
                if not 'abstract' in publication.content or not publication.content.get('abstract'):
                    continue
            if self.config.get('dataset', {}).get('with_title', False):
                if not 'title' in publication.content or not publication.content.get('title'):
                    continue
            if getattr(publication, 'cdate') is None:
                publication.cdate = getattr(publication, 'tcdate', 0)
            reduced_publication = {
                'id': publication.id,
                'cdate': publication.cdate,
                'content': {
                    'title': publication.content.get('title'),
                    'abstract': publication.content.get('abstract')
                }
            }
            unsorted_publications.append(reduced_publication)

        # If the author does not have publications, then return early
        if not unsorted_publications:
            return unsorted_publications

        dataset_params = self.config.get('dataset', {})
        minimum_pub_date = dataset_params.get('minimum_pub_date') or dataset_params.get('or', {}).get('minimum_pub_date', 0)
        top_recent_pubs = dataset_params.get('top_recent_pubs') or dataset_params.get('or', {}).get('top_recent_pubs', False)

        # If there is no minimum publication date and no recent publications constraints we return
        # all the publications in any order
        if not top_recent_pubs and not minimum_pub_date:
            return unsorted_publications

        # Sort publications in descending order based on the cdate
        sorted_publications = sorted(unsorted_publications, key=lambda pub: pub.get('cdate'), reverse=True)

        if not top_recent_pubs:
            return [publication for publication in sorted_publications if publication['cdate'] >= minimum_pub_date]

        paper_percent = 0
        if isinstance(top_recent_pubs, str) and top_recent_pubs[-1] == '%':
            paper_percent = int(top_recent_pubs[:-1]) / 100
        elif isinstance(top_recent_pubs, int):
            paper_num = top_recent_pubs

        if paper_percent:
            non_int_value = len(sorted_publications) * paper_percent
            # Use remainder to always round up decimals, then convert to int.
            # This is useful if the percentage is 10%, but the user only has 3 publications, for example.
            # 3 * 0.1 = 0.3. So we want to round up to 1 in this case.
            paper_num = int(non_int_value) + (non_int_value % 1 > 0)

        top = sorted_publications[:paper_num]
        if not minimum_pub_date:
            return top

        minimum = [publication for publication in sorted_publications if publication['cdate'] >= minimum_pub_date]

        # We need to figure out if the constraints have an OR or AND relation
        if dataset_params.get('or', False):
            # minimum and top will have the same pubs because they are sorted, however, one of them
            # is larger than the other. In OR relation we want the largest one as it would be the
            # union between the two sets
            if len(minimum) > len(top):
                return minimum
            return top

        # AND relation
        if len(minimum) > len(top):
            return top
        return minimum

    def get_profile_ids(self, group_ids=None, reviewer_ids=None):
        """
        Returns a list of all the tilde id members from a list of groups.

        Example:

        >>> get_profiles(['ICLR.cc/2018/Conference/Reviewers'])

        :param client: OpenReview Client
        :type client: Client
        :param group_ids: List of group ids
        :type group_ids: list[str]

        :return: List of tuples containing (tilde_id, email)
        :rtype: list
        """
        tilde_members = set()
        email_members = set()

        if group_ids:
            for group_id in group_ids:
                group = self.openreview_client.get_group(group_id)
                for member in group.members:
                    if '~' in member:
                        tilde_members.add(member)
                    elif '@' in member:
                        email_members.add(member.lower())

        if reviewer_ids:
            for reviewer_id in reviewer_ids:
                if '~' in reviewer_id:
                    tilde_members.add(reviewer_id)
                elif '@' in reviewer_id:
                    email_members.add(reviewer_id.lower())

        members = []
        tilde_members_list = list(tilde_members)
        profile_search_results = self.openreview_client.search_profiles(emails=None, ids=tilde_members_list, term=None) if tilde_members_list else []
        tilde_members_list = []
        for profile in profile_search_results:
            preferredEmail = profile.content.get('preferredEmail')
            # If user does not have preferred email, use first email in the emailsConfirmed list
            preferredEmail = preferredEmail or profile.content.get('emailsConfirmed') and len(profile.content.get('emailsConfirmed')) and profile.content.get('emailsConfirmed')[0]
            # If the user does not have emails confirmed, use the first email in the emails list
            preferredEmail = preferredEmail or profile.content.get('emails') and len(profile.content.get('emails')) and profile.content.get('emails')[0]
            # If the user Profile does not have an email, use its Profile ID
            tilde_members_list.append((profile.id, preferredEmail or profile.id))
        members.extend(tilde_members_list)

        email_members_list = list(email_members)
        profile_search_results = self.openreview_client.search_profiles(emails=email_members_list, ids=None, term=None) if email_members_list else {}
        email_profiles = []
        for email, profile in profile_search_results.items():
            email_profiles.append((profile.id, email))
        members.extend(email_profiles)

        invalid_members = []
        valid_members = list(set(members))
        if len(email_members):
            valid_emails = set([email_id for _, email_id in valid_members])
            for member in email_members:
                if member not in valid_emails:
                    invalid_members.append(member)

        return valid_members, invalid_members

    def exclude(self):
        exclusion_invitations = self.convert_to_list(self.config['exclusion_inv'])

        for invitation in exclusion_invitations:
            excluded_ids_by_user = defaultdict(list)
            user_grouped_edges = openreview.tools.iterget_grouped_edges(
                self.openreview_client,
                invitation=invitation,
                groupby='tail',
                select='id,head,label,weight'
            )

            for edges in user_grouped_edges:
                for edge in edges:
                    excluded_ids_by_user[edge.tail].append(edge.head)

        return excluded_ids_by_user
    
    def retrieve_expertise_helper(self, member, email):
        self.pbar.update(1)
        member_papers = self.get_publications(member)

        filtered_papers = [
            n for n in member_papers \
            if n['id'] not in self.excluded_ids_by_user[member] \
        ]

        seen_keys = set()
        filtered_papers = []
        for n in member_papers:

            paperhash = openreview.tools.get_paperhash('', n['content']['title'])

            if n['id'] not in self.excluded_ids_by_user[member] and paperhash not in seen_keys:
                filtered_papers.append(n)
                seen_keys.add(paperhash)

        return member, email, filtered_papers


    def retrieve_expertise(self):
        # if group ID is supplied, collect archives for every member
        # (except those whose archives already exist)
        use_email_ids = self.config.get('use_email_ids', False)
        group_ids = self.convert_to_list(self.config.get('match_group', []))
        reviewer_ids = self.convert_to_list(self.config.get('reviewer_ids', []))
        valid_members, invalid_members = self.get_profile_ids(group_ids=group_ids, reviewer_ids=reviewer_ids)

        self.metadata['no_profile'] = invalid_members

        print('finding archives for {} valid members'.format(len(valid_members)))

        expertise = defaultdict(list)
        futures = []
        self.pbar = tqdm(total=len(valid_members), desc='Retrieving expertise...')
        with ThreadPoolExecutor(max_workers=self.config.get('max_workers', 1)) as executor:
            for (member, email) in valid_members:
                futures.append(executor.submit(self.retrieve_expertise_helper, member, email))
        self.pbar.close()

        for future in futures:
            member, email, filtered_papers = future.result()
            member_id = email if use_email_ids else member
            if len(filtered_papers) == 0:
                self.metadata['no_publications_count'] += 1
                self.metadata['no_publications'].append(member_id)
            else:
                for paper in filtered_papers:
                    expertise[member_id].append(paper)

        return expertise
        

    def get_submissions(self):
        invitation_ids = self.convert_to_list(self.config['paper_invitation'])
        submissions = []

        for invitation_id in invitation_ids:
            submissions.extend(list(openreview.tools.iterget_notes(
                self.openreview_client, invitation=invitation_id)))

        print('finding records of {} submissions'.format(len(submissions)))
        reduced_submissions = {}
        for paper in tqdm(submissions, total=len(submissions)):
            # file_path = Path(self.submission_dir).joinpath(paper.id + '.jsonl')
            paper_id = paper.id
            reduced_submissions[paper_id] = {
                'id': paper.id,
                'content': {
                    'title': paper.content.get('title'),
                    'abstract': paper.content.get('abstract')
                }
            }
            # if not file_path.exists():
            #     with open(file_path, 'w') as f:
            #         f.write(json.dumps(paper.to_json()) + '\n')

        return reduced_submissions

    def run(self):
        dataset_dir = self.config.get('dataset', {}).get('directory')
        self.dataset_dir = Path(dataset_dir) if dataset_dir else Path('./')
        if not self.dataset_dir.is_dir():
            self.dataset_dir.mkdir()
        
        if 'exclusion_inv' in self.config:
            self.excluded_ids_by_user = self.exclude()
        else:
            self.excluded_ids_by_user = defaultdict(list)

        if 'match_group' in config or 'reviewer_ids' in self.config:
            self.archive_dir = self.dataset_dir.joinpath('archives')
            if not self.archive_dir.is_dir():
                self.archive_dir.mkdir()
            expertise = self.retrieve_expertise()
            for reviewer_id, pubs in expertise.items():
                with open(self.archive_dir.joinpath(reviewer_id), 'w') as f:
                    for paper in pubs:
                        f.write(json.dumps(paper) + '\n')
            # with open(self.root.joinpath('archive.json'), 'w') as f:
            #     json.dump(expertise, f, indent=2)

        # if invitation ID is supplied, collect records for each submission
        if 'paper_invitation' in config:
            self.submission_dir = self.dataset_dir.joinpath('submissions')
            if not self.submission_dir.is_dir():
                self.submission_dir.mkdir()
            submissions = self.get_submissions()
            with open(self.root.joinpath('submissions.json'), 'w') as f:
                json.dump(submissions, f, indent=2)
            self.metadata['submission_count'] = len(submissions.keys())

In [ ]:
# Execute this cell only if you are running the notebook in Google Colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ls '/content/gdrive/My Drive'

root = '/content/gdrive/My Drive/expertise'

In [2]:
# Execute this cell if you are running locally to specify where to place your scores
root = '/Users/cmondragonch/Documents/experiment_dir'

In [3]:
from getpass import getpass

username = input('Enter username: ')
password = getpass('Enter password: ')

client = openreview.Client(baseurl='https://openreview.net', username=username, password=password)

print('User:', getattr(client.profile, 'id', 'Guest') if client.profile else 'Guest')

User: ~Super_User1


In [4]:
config = {
    "match_group": "NeurIPS.cc/2020/Conference/Reviewers",
    "dataset": {
        "directory": root
    },
    "use_email_ids": True,
    "max_workers": 6
}

expertise = OpenReviewExpertise(client, config)
expertise.run()
with open(Path(root).joinpath('metadata.json'), 'w') as f:
    json.dump(expertise.metadata, f, indent=2)

Retrieving expertise...:   3%|▎         | 234/8585 [00:26<16:17,  8.54it/s]

KeyboardInterrupt: 